In [1]:
# -*- coding: utf-8 -*-
# This is my contribution for the GAME OFF 2018
# The theme is Hybrid
# My game is a Hybrid of a cave RPG, sort of rogue like and a match three game,
# where putting together three things of the same type gives you points or an
# upgrade.
# When judging please take into account that this is my first game, and my 
# first attempt to make such a big piece of code.
# I also did the graphics, by hand, with paint :D

# LIBRARIES
import sys
#import os
import pygame as pg

# FREE CONSTANTS
BOX_EDGE = 30       # PLAYER AND BOX SQUARES; FUDAMENTAL UNIT
GRID_WIDTH  = 21    # PLAY GRID (ODD NUMBERS)
GRID_HEIGHT = 15
BORDER_GRID = 2
BOX_COLOR = (200,100,200)
fps = 30
DELAY = 80
bkgd_color = (0, 0, 0)
internal_color = (180,180,180)
GOAL_SCORE = [100]    # SCORE GOALS FOR EACH LEVEL

# DERIVED CONSTANTS
BORDER_WIDTH = BORDER_GRID * BOX_EDGE
SCREEN_LENGTH = BOX_EDGE * (GRID_WIDTH  + 2 * BORDER_GRID)
SCREEN_HEIGHT = BOX_EDGE * (GRID_HEIGHT + 2 * BORDER_GRID)
GAME_AREA_LENGTH = SCREEN_LENGTH - 2 * BORDER_WIDTH
GAME_AREA_HEIGHT = SCREEN_HEIGHT - 2 * BORDER_WIDTH

internal_rect = pg.Rect(BORDER_WIDTH,BORDER_WIDTH,\
                        GAME_AREA_LENGTH, GAME_AREA_HEIGHT) #GAME AREA
initial_player_position = (((GRID_WIDTH - 1)//2 + BORDER_GRID) * BOX_EDGE,\
                           ((GRID_HEIGHT - 1)//2 + BORDER_GRID) * BOX_EDGE,\
                           BOX_EDGE, BOX_EDGE) #LEVEL
initial_box_position = ((initial_player_position[0] - 3 * BOX_EDGE,\
                         initial_player_position[1], BOX_EDGE, BOX_EDGE),
                        (initial_player_position[0] + 3 * BOX_EDGE,\
                         initial_player_position[1], BOX_EDGE, BOX_EDGE),
                        (initial_player_position[0],\
                         initial_player_position[1] + 3 * BOX_EDGE, BOX_EDGE, BOX_EDGE),
                        (initial_player_position[0],\
                         initial_player_position[1] - 3 * BOX_EDGE, BOX_EDGE, BOX_EDGE),
                        (initial_player_position[0] + 3 * BOX_EDGE,\
                         initial_player_position[1] - 3 * BOX_EDGE, BOX_EDGE, BOX_EDGE),
                        (initial_player_position[0] - 3 * BOX_EDGE,\
                         initial_player_position[1] + 3 * BOX_EDGE, BOX_EDGE, BOX_EDGE),
                        (initial_player_position[0] + 3 * BOX_EDGE,\
                         initial_player_position[1] + 3 * BOX_EDGE, BOX_EDGE, BOX_EDGE),) #LEVEL
PLAYER_POSITIONS = [initial_player_position] # INITIAL PLAYER POSITION FOR EACH LEVEL
BOX_POSITIONS = [initial_box_position,]       # INITIAL BOXES POSITIONS FOR EACH LEVEL

class Level():
    
    def __init__(self,level):
        self.player = Player(PLAYER_POSITIONS[level - 1])
        self.boxes = []
        self.score = 0
        for box in BOX_POSITIONS[level - 1]:
            self.boxes.append(Box(box))
        self.update_dict()
    
    def remove_box(self, box):
        self.boxes.remove(box)
        self.update_dict()
    
    def move_player(self, x, y):
        boxes = self.boxes
        player = self.player
        index_collision1 = -1
        has_moved = False
        for index, box in enumerate(boxes): # player collides with box
            if (box.x, box.y) == (player.x + x, player.y + y):
                index_collision1 = index
                break
        if index_collision1 >= 0:
            box = boxes[index_collision1]
            index_collision2 = -1
            for index, other_box in enumerate(boxes): # box collides with box or goes out
                if (box.x + x, box.y + y) == (other_box.x, other_box.y)\
                and index != index_collision1:
                    index_collision2 = index
                    break
            if  index_collision2 < 0\
            and box.x + x >= 0\
            and box.x + x < GRID_WIDTH\
            and box.y + y >= 0\
            and box.y + y < GRID_HEIGHT:
                player.move(x, y)
                box.move(x, y)
                has_moved = True
        else:
            player.move(x, y)
            has_moved = True
        self.update_dict()
        return has_moved

    def check_line(self, row, column):
        result = False
        all_kinds = ['B1', 'B2', 'B3', 'B4', 'B5']
        pos1 = self.level_dict.get((row, column))
        pos2 = self.level_dict.get((row + 1, column))
        pos3 = self.level_dict.get((row + 2, column))
        if pos1 in all_kinds and pos2 in all_kinds and pos3 in all_kinds:
            if pos1 == pos2 and pos2 == pos3 and pos1[1] != '4':
                result = True
                return result
    
    def match_line(self, row, column):
        box_to_remove = []
        for box in self.boxes:
            if box.x == row and box.y == column:
                box_to_remove.append(box)
            elif box.x == row + 2 and box.y == column:
                box_to_remove.append(box)
            elif box.x == row + 1 and box.y == column:
                if box.kind < 5:
                    box.kind += 1
        for box in box_to_remove:
            self.remove_box(box)
        self.update_dict()
            
    
    def check_vert(self, row, column):
        result = False
        all_kinds = ['B1', 'B2', 'B3', 'B4', 'B5']
        pos1 = self.level_dict.get((row, column))
        pos2 = self.level_dict.get((row, column + 1))
        pos3 = self.level_dict.get((row, column + 2))
        if pos1 in all_kinds and pos2 in all_kinds and pos3 in all_kinds:
            if pos1 == pos2 and pos2 == pos3 and pos1[1] != '4':
                result = True
        return result
    
    def match_vert(self, row, column):
        box_to_remove = []
        for box in self.boxes:
            if box.x == row and box.y == column:
                box_to_remove.append(box)
            elif box.x == row and box.y == column + 2:
                box_to_remove.append(box)
            elif box.x == row and box.y == column + 1:
                box.kind += 1
        for box in box_to_remove:
            self.remove_box(box)
        self.update_dict()
    
    def check_cube(self, row, column):
        result = False
        all_kinds = ['B1', 'B2', 'B3', 'B4', 'B5']
        pos1 = self.level_dict.get((row, column))
        pos2 = self.level_dict.get((row, column + 1))
        pos3 = self.level_dict.get((row + 1, column))
        pos4 = self.level_dict.get((row + 1, column + 1))
        if pos1 in all_kinds and pos2 in all_kinds and pos3 and pos4 in all_kinds:
            if pos1 == pos2 and pos2 == pos3 and pos3 == pos4:
                result = True
        return result
    
    def match_cube(self, row, column):
        box_to_remove = []
        for box in self.boxes:
            if box.x == row + 1 and box.y == column + 1:
                box_to_remove.append(box)
            elif box.x == row and box.y == column + 1:
                box_to_remove.append(box)
            elif box.x == row and box.y == column:
                box.kind += 1
            elif box.x == row + 1and box.y == column:
                if self.level_dict.get((row, column))[1] != '4':
                    box.kind += 1
                else:
                    box_to_remove.append(box)
        for box in box_to_remove:
            self.remove_box(box)
        self.update_dict()
    
    def check_match(self):
        for row in range(15):
            for column in range(31):
                if row + 1 < 15:
                    if column + 1 < 31:
                        if self.check_cube(row, column):
                            self.match_cube(row, column)
                    if row + 2 < 15:
                        if self.check_line(row, column):
                            self.match_line(row, column)
                    if column + 2 < 31:
                        if self.check_vert(row, column):
                            self.match_vert(row, column)
    
    def update_dict(self):
        self.level_dict = {}
        self.level_dict[(self.player.x, self.player.y)] = "P"
        for box in self.boxes:
            self.level_dict[(box.x, box.y)] = "B" + str(box.kind)

class Box():
    def __init__(self,rectangle):
        self.rectangle = pg.Rect(rectangle)
        self.x = (rectangle[0] - BORDER_WIDTH) // BOX_EDGE
        self.y = (rectangle[1] - BORDER_WIDTH) // BOX_EDGE
        self.kind = 4
    
    def move(self, x, y):
        self.x = self.x + x
        self.y = self.y + y
        self.rectangle = pg.Rect(self.x * BOX_EDGE + BORDER_WIDTH,\
                                 self.y * BOX_EDGE + BORDER_WIDTH,\
                                 self.rectangle[2],self.rectangle[3])

class Player():
    def __init__(self,rectangle):
        self.rectangle = pg.Rect(rectangle)
        self.x = (rectangle[0] - BORDER_WIDTH) // BOX_EDGE
        self.y = (rectangle[1] - BORDER_WIDTH) // BOX_EDGE
    
    def move(self, x, y):
        self.x = self.x + x
        self.y = self.y + y
        self.x = max(self.x, 0)
        self.x = min(self.x, GRID_WIDTH - 1)
        self.y = max(self.y, 0)
        self.y = min(self.y, GRID_HEIGHT - 1)
        self.rectangle = pg.Rect(self.x * BOX_EDGE + BORDER_WIDTH,\
                                 self.y * BOX_EDGE + BORDER_WIDTH,\
                                 self.rectangle[2],self.rectangle[3])
    
class GameArea():
    def __init__(self):
        self.rectangle = pg.Rect(internal_rect)
        self.current_level = 1
        self.level = Level(self.current_level)
        self.goal_score = GOAL_SCORE[self.current_level - 1]
        self.score = 0
        
        
    def restart_level(self):
        self.level = Level(self.current_level)
    
    def next_level(self):
        self.current_level += 1
        self.goal_score = GOAL_SCORE[self.current_level - 1]
        self.level = Level(self.current_level)

class Screen():
    def __init__(self, length, height):
        self.surf = pg.display.set_mode((length, height), )
        self.height = height
        self.length = length
        self.area = GameArea()

class Application():
    def __init__(self):
        self.run = True
        self._image_library = {}
        self.clock = pg.time.Clock()
        pg.display.set_caption("Maze Match")
        pg.display.set_icon(self.get_image('Maze.ico'))
        self.screen = Screen(SCREEN_LENGTH, SCREEN_HEIGHT)
    
    def ask_exit(self):
        return True
    
    def get_image(self,path):
        image = self._image_library.get(path)
        if image == None:
            image = pg.image.load(path)
            self._image_library[path] = image
        return image
    
    def main(self):
        while self.run:
            self.clock.tick(fps)
            self.update()
            self.draw()

    def draw(self):
        pg.draw.rect(self.screen.surf, internal_color, self.screen.area.rectangle)
        for i, box in enumerate(self.screen.area.level.boxes):
            self.screen.surf.blit(self.get_image('Box' + str(box.kind) + '.bmp'),\
                                  box.rectangle)
        self.screen.surf.blit(self.get_image('Player.bmp'), self.screen.area.level.player.rectangle)
        pg.display.flip()
        
    def update(self):
        for event in pg.event.get():
            if event.type == pg.QUIT:
                if self.ask_exit(): self.run = False
            elif event.type == pg.KEYDOWN and event.key == pg.K_ESCAPE:
                if self.ask_exit(): self.run = False
            elif event.type == pg.KEYDOWN and event.key == pg.K_r:
                 self.screen.area.restart_level()
            elif event.type == pg.KEYDOWN and event.key == pg.K_t:
                print(self.screen.area.level.level_dict)
                print(self.screen.area.level.level_dict.get((9, 7)))
                print(self.screen.area.level.level_dict.get((9 + 1, 7)))
                print(self.screen.area.level.level_dict.get((9 + 2, 7)))
                #raise Exception('Exception raised by user')
        
        keys = pg.key.get_pressed()
        
        level = self.screen.area.level
        count_movements = 0
        
        if keys[pg.K_a] or keys[pg.K_LEFT]:  # to move left
            if level.move_player(-1, 0):
                count_movements += 1
        if keys[pg.K_d] or keys[pg.K_RIGHT]: # to move right
            if level.move_player(1, 0):
                count_movements += 1
        if keys[pg.K_w] or keys[pg.K_UP]: #to move up
            if level.move_player(0, -1):
                count_movements += 1
        if keys[pg.K_s] or keys[pg.K_DOWN]: #to move down
            if level.move_player(0, 1):
                count_movements += 1
        if count_movements >= 1:
            pg.time.delay(DELAY)
        
        level.check_match()
        
if __name__ == '__main__':
    pg.init()
    try:
        app = Application()
        app.main()
    except(Exception):
        print("Unhandled exception in file " + \
              str(sys.exc_info()[2].tb_frame.f_code.co_filename) + \
              " on line " + str(sys.exc_info()[2].tb_lineno) + ".\nGot a " + \
              str(sys.exc_info()[0]) + " while running.\nExplanation: " + \
              str(sys.exc_info()[1]) + "\nThe application will now quit.")
        pg.quit()
    pg.quit()


ModuleNotFoundError: No module named 'pygame.base'

In [6]:
def main():
    pass